In [1]:
import requests
from multiprocessing import Pool
from tqdm import tqdm
import urllib3
urllib3.disable_warnings()

In [2]:
badWords = [
    'is available to buy',
    'is available for sale'
]

In [3]:
def checkDomain(domain):
    try:
        domain = domain.strip()
        url = f'http://{domain}/'
        res = requests.get(url,timeout=1, verify=False)
        source = res.text.lower()
        if res.status_code == 200:
            for item in badWords:
                if item in source:
                    return None#f"#{domain} - {item}"
            return domain
        else:
            return None # f"#{domain} - {res.status_code}"            
    except Exception as e:
        return None # f"#{domain} - {type(e)}"

In [5]:
if __name__ == '__main__':
    domains = open('disposable_email_blocklist.conf','r').readlines()
    print(f'Loaded {len(domains)} domains')
    with Pool(35) as p:
        scanned_domains = p.map(checkDomain, domains)
        
    valid_domains = list(filter(None,scanned_domains))
    
    valid_domains.sort()
    
    with open ('disposable_email_blocklist.conf','w') as f:
        for item in valid_domains:
            f.write(f'{item}\n')

    print(f'Writing {len(valid_domains)} domains')

Writing 1188 domains
